In [ ]:
import math
import numpy as np
import matplotlib.pyplot as pp
%matplotlib inline

inf = float('inf')

In [ ]:
x = np.linspace(0, 10, 100)
y = np.sin(x) + np.sin(2.1*x) + 0.5*np.sin(6.3*x) - 0.5

In [ ]:
pp.plot(X, y)
pp.axhline(0)

In [ ]:
def D(x):
    ddx = np.zeros_like(x)
    ddx[1: -1] = x[2:] - x[:-2]
    ddx[0], ddx[-1] = ddx[1], ddx[-2]
    return ddx

def thresh(x):
    return np.array([e > 0 and inf or 0 for e in x])

y2 = y/np.abs(D(y))

pp.figure(figsize=(40, 40))
ax = pp.gca()
ax.set_aspect(1)
pp.ylim(-10, 10)
pp.plot(y)
pp.plot(y2)
pp.axhline(0, color="blue")
# pp.plot(foo)
# pp.axhline(1, color="orange")

In [ ]:
def compute_edt(d):
    v, z = find_parabolas(d)
    d = march_parabolas(d, v, z)
    return np.sqrt(d)

def find_parabolas(d):
    l = len(d)
    vx, vy = np.zeros(l + 1), np.zeros(l + 1)
    zx = np.full(l + 1, inf)
    j = 0
    vx[0], vy[0] = 0, d[0]
    zx[0] = -inf
    for i in range(1, len(d)):
#         print(f"i: {i}, j:{j}")
        px, py = vx[j], vy[j]
        qx, qy = i, d[i]
#         print(f"  v[{j}]: ({px}, {py}), q: ({qx}, {qy})")
        sx = intersect_parabolas(px, py, qx, qy)
#         print(f"  sx: {sx}, zx[{j}]: {zx[j]}")
        while j > 0 and sx <= zx[j]:
            j -= 1
            px, py = vx[j], vy[j]
            sx = intersect_parabolas(px, py, qx, qy)
#             print(f"    j: {j}, p: ({px}, {py}), sx: {sx}")
        j += 1
#         print(f"  v[{j}] = ({qx}, {qy}), zx[{j}] = {sx}")
        vx[j], vy[j] = qx, qy
        zx[j] = sx
    
    return (vx, vy), zx

def intersect_parabolas(px, py, qx, qy):
    if py == qy: return (px + qx)/2
    else: return ((qy + qx*qx) - (py + px*px))/(2*qx - 2*px)

def march_parabolas(d, v, zx):
    d = np.zeros_like(d)
    vx, vy = v
    j = 0
    for i in range(len(d)):
        while i > zx[j + 1]: j += 1
#         print(f"i: {i}, vx[{j}]: {vx[j]}")
        dx = i - vx[j]
        d[i] = dx*dx + vy[j]
    return d

pp.figure(figsize=(40, 40))
pp.gca().set_aspect(1)
pp.ylim(-10, 10)

pp.plot(4*y)

edtp = compute_edt(thresh( y))
edtn = compute_edt(thresh(-y))
sdf = np.maximum(edtp - 0.5, 0) - np.maximum(edtn - 0.5, 0)
pp.axhline(0)
pp.plot(sdf)

# yn = 4*y/np.abs(D(y))
# yn = np.array([abs(y) < 2 and y or math.copysign(inf, y) for y in yn])
# pp.plot(yn)

# edt2 = compute_edt(np.abs(yn**2))*np.sign(yn)
# pp.plot(edt2, "o-")

In [ ]:
pp.figure(figsize=(40, 40))
pp.gca().set_aspect(1)
pp.axhline(0)
pp.ylim(-10, 10)

# pp.plot(sdf)
edtp = compute_edt(thresh( y))
pp.plot(edtp)
pp.plot(compute_edt(edtp**2))

# sdf2 = np.array(sdf)
# sdf2 = np.copysign(compute_edt(np.abs(sdf2) + 0.5), sdf2)
# pp.plot(sdf2)